In [1]:
!pip3 install -q towhee==1.0.0rc1 pymilvus dill --user


In [2]:
COLLECTION_NAME="termsofservice"
HOST="10.113.14.98"
PORT="19530"

In [3]:
from milvus import default_server
from pymilvus import connections, utility

connections.connect(host=HOST, port=PORT)

connections

In [4]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility


In [5]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)
# object should be inserted in the format of (title, date, location, speech embedding)
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768)
]
schema = CollectionSchema(fields=fields, description='search text')
collection = Collection(name=COLLECTION_NAME, schema=schema)
index_params = {
    'metric_type': "L2",
    'index_type': "IVF_FLAT",
    'params': {"nlist": 2048}
}
collection.create_index(field_name='embedding', index_params=index_params)


Status(code=0, message='')

In [8]:
import towhee

bert_op = towhee.ops.text_embedding.transformers(model_name='bert-base-uncased').get_op()

In [9]:
def readfile(filename):
    #open text file in read mode
    text_file = open(filename, "r")

    #read whole file to a string
    data = text_file.read()

    #close file
    text_file.close()
    embedding = bert_op(data)
    return embedding[0]


In [11]:
import pandas as pd
column_names=["embedding"]

data = [ [readfile("tos.txt")], 
         [readfile("tos1.txt")], 
         [readfile("tos2.txt")]]
df=pd.DataFrame(data, columns=column_names)
df.head()



,embedding
0,"[-0.05431702, -0.08775247, -0.07701182, -0.160..."
1,"[-0.0014222626, -0.19510998, -0.004679618, 0.0..."
2,"[0.21410906, -0.23142844, -0.103328414, -0.030..."


In [12]:
df.values.tolist()

[[array([-5.43170199e-02, -8.77524689e-02, -7.70118237e-02, -1.60241917e-01,
          2.41497979e-01, -7.26048172e-01,  6.08089924e-01,  6.58015788e-01,
          2.81623155e-01, -5.81767440e-01, -2.41878778e-01,  2.85535753e-01,
          1.22068916e-03,  1.04648754e-01,  2.97260284e-01,  6.44459426e-02,
         -4.37864423e-01,  7.76952326e-01,  2.03693718e-01,  2.06438020e-01,
         -2.44132936e-01, -8.31654847e-01,  4.44627285e-01, -1.92655578e-01,
          1.99311569e-01, -5.23372379e-04, -5.24483263e-01, -2.21357360e-01,
         -6.43513262e-01, -4.48804975e-01, -8.06863829e-02,  5.18662274e-01,
         -4.28608596e-01, -5.80957055e-01,  2.15977445e-01, -6.49146140e-01,
         -3.86863090e-02,  7.99465850e-02,  1.65299084e-02,  3.08759570e-01,
         -2.80600995e-01,  9.52115189e-03,  4.77030098e-01, -2.35012323e-01,
          7.63794407e-02,  5.42070307e-02, -3.42405295e+00, -1.45097136e-01,
         -2.09071726e-01, -2.54826963e-01,  1.29118964e-01, -1.56936795e-01,

In [13]:

from towhee import ops, pipe, DataCollection

p =  (pipe.input('df')
            .flat_map('df', 'data', lambda df: df.values.tolist())
            .map('data', 'res', ops.ann_insert.milvus_client(host=HOST, 
                                                            port=PORT,
                                                            collection_name=COLLECTION_NAME))
             .output('res')
)

In [14]:
p(df)

In [15]:
collection.load()
collection.num_entities

3

In [ ]:
collection.load()
collection.num_entities

In [16]:
import numpy as np

search_pipe = (pipe.input('query')
                    .map('query', 'vec', ops.text_embedding.dpr(model_name="distilbert-base-cased"))
                    .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
                    .flat_map('vec', ('id', 'score'), ops.ann_search.milvus_client(host=HOST, 
                                                                                   port=PORT,
                                                                                   collection_name=COLLECTION_NAME))  
                    .output('query', 'id', 'score')
               )

In [19]:
res = search_pipe.batch(['Fogbender Last updated', 'support@fogbender.com'])
for re in res:
    DataCollection(re).show()

query,id,score
Fogbender Last updated,441360469201748342,234.95936584472656
Fogbender Last updated,441360469201748376,235.34954833984375
Fogbender Last updated,441360469201748359,237.30453491210938


query,id,score
support@fogbender.com,441360469201748342,235.1484832763672
support@fogbender.com,441360469201748376,235.488037109375
support@fogbender.com,441360469201748359,237.4297637939453


In [ ]:
res = search_pipe('Fogbender Last modified')
DataCollection(res).show()